# Senstivity study for the `Courior Only` model - `RMSS2_V2.2C`

### This model is based on work by: Saltelli
Great book (game changer for myself) --> https://www.amazon.com/Sensitivity-Analysis-Practice-Assessing-Scientific/dp/0470870931/ref=sr_1_1?dchild=1&qid=1626197470&refinements=p_27%3AAndrea+Saltelli&s=books&sr=1-1&text=Andrea+Saltelli <br>
Sensitivity Analysis in Practice: A Guide to Assessing Scientific Models 1st Edition <br>
by Andrea Saltelli (Author), Stefano Tarantola (Author), Francesca Campolongo (Author), Marco Ratto (Author)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import pyNetLogo

#Import the sampling and analysis modules for a Sobol variance-based
#sensitivity analysis
from SALib.sample import saltelli
from SALib.analyze import sobol

### Variables - format:[variable-name, [low, step, high]
The following varaibles are avalible for use. Exersized variables bold <br>
["charger-network" true false]<br>
["has-charge-station" [0 0.01 1]]<br>
**["service-range" [1 1 170]]<br>**
**["chance-of-medical-event" [0 1.0E-4 0.01]]**<br>
**["chance-of-standard" [0 0.01 1]]**<br>
_["chance-of-emergency" [0 0.01 1]] - caluclated in netlogo (1-chance-of-emergency)_<br>
["activate-hospitals" true false]<br>
["drone-types" "444444444444444444444"] <br>
["replacebattery?" true false]<br>
["maxpayloadweight" [4 1 150]]<br>
**["meanpayloadweight" [1 1 150]]**<br>
**["startup-costs" [50000 10000 1000000]]**<br>
**["maintenace-cost-per-hour" [0 10 1000]]**<br>
["discharge_adjustment" [0 1 400]]<br>
**["chance_of_typea" [0 0.01 1]]**<br>
**["chance_of_typeb" [0 0.01 1]]**<br>
**["chance_of_typec" [0 0.01 1]]**<br>
**["number_of_drones" [1 1 30]]**<br>

### Responses 
- mean avgResponseTimeStandard
- mean avgResponseTimeEmergency
- length tmpList (queue Length) - no used now
- fitnessFunc
- totalOperatingCosts

In [2]:
problem = {
      'num_vars': 11,
      'names': ['random-seed',
                'service-range',
                'chance-of-medical-event',
                'chance-of-standard',
                'meanpayloadweight',
                'startup-costs',
                'maintenace-cost-per-hour',
                'chance_of_typea',
                'chance_of_typeb',
                'chance_of_typec',
                'number_of_drones'],
      'bounds': [[1, 100000],
                 [20, 120],
                 [1e-4, .01],
                 [0, 1],
                 [2, 20],
                 [0, 2e6],
                 [20, 200],
                 [0,1],
                 [0,1],
                 [0,1],
                 [3,30]]
    }
problem

{'num_vars': 11,
 'names': ['random-seed',
  'service-range',
  'chance-of-medical-event',
  'chance-of-standard',
  'meanpayloadweight',
  'startup-costs',
  'maintenace-cost-per-hour',
  'chance_of_typea',
  'chance_of_typeb',
  'chance_of_typec',
  'number_of_drones'],
 'bounds': [[1, 100000],
  [20, 120],
  [0.0001, 0.01],
  [0, 1],
  [2, 20],
  [0, 2000000.0],
  [20, 200],
  [0, 1],
  [0, 1],
  [0, 1],
  [3, 30]]}

In [3]:
type(problem)

dict

In [4]:
n = 20
param_values = saltelli.sample(problem, n, calc_second_order=True)
#param_values

/opt/tljh/user/lib/python3.7/site-packages/SALib/sample/saltelli.py:80: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (20) is equal to `2^n`.
        
  warnings.warn(msg)


In [5]:
# start the cluster -> ipcluster start -n 2
import ipyparallel
client = ipyparallel.Client()
client.ids

[0, 1, 2]

In [6]:
direct_view = client[:]

In [9]:
import os

#Push the current working directory of the notebook to a "cwd" variable on the engines that can be accessed later
direct_view.push(dict(cwd=os.getcwd()))

<AsyncResult: _push>

In [11]:
%%px

import os
os.chdir(cwd)

import pyNetLogo
import pandas as pd
#netlogo = pyNetLogo.NetLogoLink(gui=False)
path = "/srv/share/NetLogo_6.2.0"
ver = "6.2"
netlogo = pyNetLogo.NetLogoLink(netlogo_home = path, netlogo_version=ver, gui=False)
netlogo.load_model('RMSS2_V2.2C')

### Note: Had to move the 'problem' dict inside the func... _probably an issue with it not being published to the cluster._

In [12]:
def simulation(experiment):
    
    problem = {
      'num_vars': 11,
      'names': ['random-seed',
                'service-range',
                'chance-of-medical-event',
                'chance-of-standard',
                'meanpayloadweight',
                'startup-costs',
                'maintenace-cost-per-hour',
                'chance_of_typea',
                'chance_of_typeb',
                'chance_of_typec',
                'number_of_drones'],
      'bounds': [[1, 100000],
                 [20, 120],
                 [1e-4, .01],
                 [0, 1],
                 [2, 20],
                 [0, 2e6],
                 [20, 200],
                 [0,1],
                 [0,1],
                 [0,1],
                 [3,30]]
    }

    #Set the input parameters
    for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(experiment[i]))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, experiment[i]))

    netlogo.command('setup')
    #Run for XXXX ticks and return the response values
    response = netlogo.repeat_report(['mean avgResponseTimeStandard','mean avgResponseTimeEmergency', 'fitnessFunc', 'totalOperatingCosts'], 100)

    results = pd.Series([response['avgResponseTimeStandard'].values.mean(),
                         response['avgResponseTimeEmergency'].values.mean(),
                         response['fitnessFunc'].values.mean(),
                         response['totalOperatingCosts'].values.max],
                        index=['Average Standard - RT', 'Average Emergnency - RT', 'fitness', 'Operating Costs'])

    return results

In [13]:
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values))

CompositeError: one or more exceptions from call to method: simulation
[0:apply]: NetLogoException: org.nlogo.nvm.RuntimePrimitiveException: Can't find the mean of a list with no numbers: [].
[1:apply]: NetLogoException: org.nlogo.nvm.RuntimePrimitiveException: Can't find the mean of a list with no numbers: [].
[2:apply]: NetLogoException: org.nlogo.nvm.RuntimePrimitiveException: Can't find the mean of a list with no numbers: [].
[0:apply]: NetLogoException: org.nlogo.nvm.RuntimePrimitiveException: Can't find the mean of a list with no numbers: [].
.... 476 more exceptions ...

In [ ]:
results.to_csv('data/RMSS2_V2.2C_2Sobol_parallel.csv')

In [ ]:
results.head(5)